## Download necessary stuff

In [ ]:
# Download the annotations and stuff

!git clone https://github.com/leolani/cltl-face-all

!wget https://raw.githubusercontent.com/leolani/cltl-face-all/master/examples/smaller-datatsets-jsons/datasets.json


!gdown --id 1-2LeHC_5Cm2gWWT6vBrVhp8jorbjkN1_
!unzip visual-features.zip
!rm visual-features.zip

!gdown --id 16ck7plW9v9eSHGCs5wuB2AhhufPRt3Wi
!unzip smaller-dataset.zip
!rm smaller-dataset.zip

## Read pre-computed visual features and annotations

In [ ]:
import json
import numpy as np
import os
import json
import av
import cv2
import random
from glob import glob
import pickle
from datetime import datetime, timedelta

VISUAL_FEATURES_PATH = 'visual-features/'
VIDEOS_PATH = "smaller-dataset/"


with open('datasets.json', 'r') as stream:
    datasets = json.load(stream)

datasets = datasets['large']

visual_features = glob('visual-features/*.npy')
visual_features = {os.path.basename(vf).split('.npy')[0] : np.load(vf, allow_pickle=True).item() for vf in visual_features}

with open('friends-time/friends-time.pkl', 'rb') as stream:
    friends_time = pickle.load(stream)

## Run on the images

In [55]:
import os
import av
import numpy as np
import cv2
from tqdm.notebook import tqdm
import time
import csv
from glob import glob
import numpy as np
import shutil

THRESHOLDS = {'face': 0.8, 'angle': 1.15}

predefined_faces = {}
for path in glob(os.path.join('./cltl-face-all/your-faces/*/*.npy')):
    name = path.split('/')[-2]
    predefined_faces[name] = np.load(path)

def calc_angle_distance(emb1, emb2):
    """Calculate the angle (radian) distance between the embeddings."""
    return np.arccos(np.clip((emb1 @ emb2.T), -1, 1))

def get_dias(list_of_diautts):
    return sorted(list(set([diautt.split('_')[0] for diautt in list_of_diautts])))


In [56]:
for DATASET in tqdm(['train', 'dev', 'test']):
    dataset = datasets[DATASET]
    diautts_ = list(dataset.keys())
    
    dias = get_dias(diautts_)
    diautts_ = {dia: [diautt for diautt in diautts_ if dia in diautt] for dia in dias}

    for dia, diautts in tqdm(diautts_.items()):
        shutil.rmtree(os.path.join('data', dia), ignore_errors=True)
        os.makedirs(os.path.join('data', dia, 'image'), exist_ok=True)
        os.makedirs(os.path.join('data', dia, 'text'), exist_ok=True)
        os.makedirs(os.path.join('data', dia, 'audio'), exist_ok=True)

        # annots = {diautt: datasets['train'][diautt] for diautt in diautts}
        # vids_path = {diautt: os.path.join(VIDEOS_PATH, diautt) +  '.mp4' for diautt in diautts}
        # features_path = {diautt: os.path.join(VISUAL_FEATURES_PATH, diautt) +  '.npy' for diautt in diautts}
        # features = {diautt: np.load(path, allow_pickle=True).item() for diautt, path in features_path.items()}

        image_grmc = []
        chat = []
        for diautt in tqdm(diautts):
            annot = datasets[DATASET][diautt] 
            vis = visual_features[diautt]
            vidpath = os.path.join(VIDEOS_PATH, diautt) + '.mp4'

            season = annot['Season']
            episode = annot['Episode']
            emotion = annot['Emotion']
            sentiment = annot['Sentiment']
            starttime = annot['StartTime']
            endttime = annot['EndTime']
            utterance = annot['Utterance']
            speaker = annot['Speaker']

            hours, minutes, seconds = starttime.split(':')
            seconds, milliseconds = seconds.split(',')

            hours, minutes, seconds, milliseconds = int(hours), int(minutes), int(seconds), int(milliseconds)

            time_datetime = friends_time[season][episode] + timedelta(hours=hours, minutes=minutes, seconds=seconds)
            time_unix = time.mktime(time_datetime.timetuple())
            time_unix_ms = int(time_unix*1000 + milliseconds)

            chat.append([speaker, utterance, time_unix_ms])

            aud = diautt + '.wav'
            !ffmpeg -i $vidpath -q:a 0 -map a data/$dia/audio/$aud

            container = av.open(vidpath)
            fps = float(container.streams.video[0].average_rate)
            for frame in container.decode(video=0):
                idx = frame.index
                numpy_RGB = np.array(frame.to_image())
                numpy_BGR = cv2.cvtColor(numpy_RGB, cv2.COLOR_RGB2BGR)
                impath = os.path.join('data', dia, 'image', diautt + f'_frame{str(idx).zfill(5)}.jpg')
                cv2.imwrite(impath, numpy_BGR)

                features = vis[idx]
                # Assume that there is only one unique face per frame.
                for feat in features:
                    to_append = {}

                    age = float(feat['age'])
                    gender = float(feat['gender'])
                    bbox = feat['bbox']

                    bbox, faceprob = [int(round(bb)) for bb in bbox[:4]], float(bbox[-1])

                    embedding = feat['embedding']
                    landmark = feat['landmark']

                    if bbox[-1] < THRESHOLDS['face']:
                        continue

                    if speaker not in list(predefined_faces.keys()):
                        continue

                    embedding.reshape(1, 512)

                    dists = {key: calc_angle_distance(embedding, val) for key, val \
                                in predefined_faces.items()}

                    if dists[speaker] < THRESHOLDS['angle']:
                        to_append['files'] = impath
                        to_append['bbox'] = bbox
                        to_append['faceprob'] = round(faceprob, 3)
                        to_append['speaker'] = speaker
                        to_append['age'] = round(age, 3)
                        to_append['gender'] = round(gender, 3)

                        image_grmc.append(to_append)

        with open(os.path.join('data', dia, 'text', f'{dia}.csv'), 'w') as stream:
            stream.write('speaker,utterance,time\n')

            for line in chat:
                speaker, utterance, time_unix_ms = line
                stream.write(speaker)
                stream.write(',')
                stream.write(f"\"{utterance}\"")
                stream.write(',')
                stream.write(str(time_unix_ms))
                stream.write('\n')    

        with open(os.path.join('data', dia, 'image.json'), 'w') as stream:
            json.dump(image_grmc, stream)

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzmq --enable-libzvbi --enable-lv2 --enable-omx --enable-openal --enable-o

flite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzmq --enable-libzvbi --enable-lv2 --enable-omx --enable-openal --enable-opencl --enable-opengl --enable-sdl2 --enable-libdc1394 --enable-libdrm --enable-libiec61883 --enable-nvenc --enable-chromaprint --enable-frei0r --enable-libx264 --enable-shared
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 / 

lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzmq --enable-libzvbi --enable-lv2 --enable-omx --enable-openal --enable-opencl --enable-opengl --enable-sdl2 --enable-libdc1394 --enable-libdrm --enable-libiec61883 --enable-nvenc --enable-chromaprint --enable-frei0r --enable-libx264 --enable-shared
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100


e-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzmq --enable-libzvbi --enable-lv2 --enable-omx --enable-openal --enable-opencl --enable-opengl --enable-sdl2 --enable-libdc1394 --enable-libdrm --enable-libiec61883 --enable-nvenc --enable-chromaprint --enable-frei0r --enable-libx264 --enable-shared
  libavutil      56. 31.100 / 56. 31.100
  libavcode

...

a-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzmq --enable-libzvbi --enable-lv2 --enable-omx --enab

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzmq --enable-libzvbi --enable-lv2 --enable-omx --enabl

wav, to 'data/dia549/audio/dia549_utt2.wav':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    ISFT            : Lavf58.29.100
    Chapter #0:0: start 0.000000, end 9.550000
    Metadata:
      title           : 00:02:29.232
    Stream #0:0(eng): Audio: pcm_s16le ([1][0][0][0] / 0x0001), 48000 Hz, 5.1, s16, 4608 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      encoder         : Lavc58.54.100 pcm_s16le
size=    5376kB time=00:00:09.55 bitrate=4608.1kbits/s speed= 250x    
video:0kB audio:5376kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.001853%
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample -

TypeError: Expected Ptr<cv::UMat> for argument 'src'